In [1]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import preprocessing
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import pandas as pd
import phate
import math
import random
import gc
import scprep
from datetime import datetime, time
from matplotlib.animation import ImageMagickWriter
import matplotlib.animation as animation
import zipfile
from urllib.request import urlopen
import scipy.stats as st
from scipy.stats import norm
from scipy.stats import gaussian_kde
from scipy.stats import kde
from scipy.stats import binned_statistic
from scipy.stats import f_oneway
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams['pdf.fonttype'] = 42
print(sns.__version__)
from anndata import AnnData
import scanpy as sc
from delve import *
import anndata as ad
from sklearn.preprocessing import MinMaxScaler
from kh import sketch
from sklearn.cluster import KMeans
import umap
print(sc.__version__)
today = datetime.now().strftime("%m%d%Y-%H%M")

0.11.2
1.9.1


In [2]:
#Read back in the subsampled adata file
adata_save_path = r'Z:\Garrett\4i_data\112723_RPE_31day_Etop\experiment\cell_data\standard_trimmed_noPSTAT5_adata_sub.h5ad'
standard_trimmed_noPSTAT5_adata_sub = anndata.read_h5ad(adata_save_path)

c:\Users\gases\.conda\envs\python_3_7\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
# Adjusted list comprehension to exclude var_names containing "total"
columns_to_keep = [name for name in standard_trimmed_noPSTAT5_adata_sub.var_names if "total_nuc_protein" not in name]

# Selecting the data with only the columns to keep
standard_trimmed_noPSTAT5_noTotal_adata_sub = standard_trimmed_noPSTAT5_adata_sub[:, columns_to_keep]


In [4]:
def laplacian_score_fs(adata = None,
                    k: int  = None,
                    n_jobs: int  = -1):

    X, feature_names, obs_names = parse_input(adata)
    W = construct_affinity(X = X, k = k, n_jobs = n_jobs)
    scores = laplacian_score(X = X, W = W)
    predicted_features = pd.DataFrame(scores, index = feature_names, columns = ['laplacian_score'])
    predicted_features = predicted_features.sort_values(by = 'laplacian_score', ascending = True)

    return predicted_features 

In [5]:
#l_score_fullest = laplacian_score_fs(adata_fullest_sub, k = 200)
l_score_standard = laplacian_score_fs(standard_trimmed_noPSTAT5_adata_sub, k = 100)
#l_score_normalized = laplacian_score_fs(normalized_trimmed_noPSTAT5_adata_sub, k = 200)

In [6]:
len(l_score_standard)

84

In [7]:
l_score_standard.index[:46]

Index(['nuc_area', 'R2_pp65_total_nuc_protein', 'R5_p38_total_nuc_protein',
       'R1_p16_total_nuc_protein', 'R4_PARP1_total_nuc_protein',
       'R3_p53_total_nuc_protein', 'R5_STAT5_total_nuc_protein',
       'R7_pp38_total_nuc_protein', 'R6_pSTAT3_total_nuc_protein',
       'R2_IL6_total_nuc_protein', 'R7_GATA4_total_nuc_protein',
       'R1_Jak2_total_nuc_protein', 'R6_p65_total_nuc_protein',
       'minor_axis_length', 'R0_Rb_total_nuc_protein',
       'R3_PLAUR_total_nuc_protein', 'R5_p38_nuc_mean', 'R0_pRb_nuc_mean',
       'major_axis_length', 'R4_IL8_total_nuc_protein',
       'R0_pRb_total_nuc_protein', 'R0_pRb_ratio', 'R1_p21_total_nuc_protein',
       'R0_DNA_total_nuc_protein', 'R6_pSTAT3_ring_mean', 'R2_IL6_ring_mean',
       'R6_p65_ring_mean', 'R1_p21_nuc_mean', 'R6_pSTAT3_nuc_mean',
       'R0_Rb_ratio', 'R5_p38_ring_mean', 'R6_p65_nuc_mean', 'R2_IL6_nuc_mean',
       'R1_p21_ratio', 'R0_Rb_nuc_mean', 'R7_GATA4_nuc_mean',
       'R1_p16_nuc_mean', 'R5_p38_ratio', 'R2

Code below taking inputs from Slingshot processing (Done in R) and any provided Clustering strategy.
Phate must be run and the PHATE coordinates saved using methods from the PHATE plotting notebook.
Produces a graph that plots specified clusters and overlays the Lineage information provided by Slingshot.

In [8]:
standard_trimmed_noPSTAT5_adata_sub.obs

,label,well,sample_ID,treatment,kmeans_clusters_k4,kmeans_clusters_k5,kmeans_clusters_k6,kmeans_clusters_k7,kmeans_clusters_k8,kmeans_clusters_k9,...,gmm_clusters_k6,gmm_clusters_k7,gmm_clusters_k8,gmm_clusters_k9,gmm_clusters_k10,gmm_clusters_k11,gmm_clusters_k12,Pseudotime_Lineage1,Pseudotime_Lineage2,Pseudotime_Lineage3
0,1772,C02,1,control,1,0,2,2,0,2,...,5,5,2,8,9,9,9,0.013550,0.013909,0.013848
1,1773,C02,1,control,1,2,3,3,5,3,...,5,5,2,8,9,9,9,0.000525,0.000525,0.000525
2,28,B02,1,control,3,1,5,4,6,8,...,2,2,7,7,7,7,2,0.052377,NaN,NaN
3,1775,C02,1,control,1,2,3,3,5,3,...,5,5,2,8,9,9,9,0.000132,0.000132,0.000132
4,2090,B02,1,control,1,2,3,3,5,3,...,5,5,2,8,9,9,9,0.008607,0.008538,0.008666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15595,965,E05,14,31day,2,0,0,5,2,5,...,5,6,0,2,2,2,11,0.019299,NaN,0.023034
15596,966,E05,14,31day,2,0,2,2,0,2,...,0,0,6,3,3,3,3,NaN,0.020898,NaN
15597,969,E05,14,31day,2,0,0,5,2,2,...,0,6,0,2,2,2,11,0.019141,NaN,0.022048
15598,974,E05,14,31day,2,3,4,0,4,0,...,2,6,0,2,7,7,2,NaN,0.033849,NaN


In [29]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pptx import Presentation
from pptx.util import Inches
import io

# Example pseudotime_cutoffs dictionary
pseudotime_cutoffs = {
    1: None,  # No cutoff for Lineage 1
    2: 0.02,  # Cutoff for Lineage 2
    3: None   # No cutoff for Lineage 3
    # Add more lineages and their cutoffs if needed
}

# Initialize PowerPoint Presentation and slide layout
prs = Presentation()
slide_layout = prs.slide_layouts[5]  # Choose a blank slide layout

# Define a function to add the slide with the plot image
def add_slide_with_plot(prs, img_data, title, legend_img_data=None):
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.title.text = title
    img_stream = io.BytesIO(img_data.read())
    img_stream.seek(0)
    pic = slide.shapes.add_picture(img_stream, Inches(1), Inches(1), width=Inches(8))

    if legend_img_data:
        legend_stream = io.BytesIO(legend_img_data.read())
        legend_stream.seek(0)
        slide.shapes.add_picture(legend_stream, Inches(9), Inches(1), width=Inches(2.5))

# Plotting loop
var_names = standard_trimmed_noPSTAT5_adata_sub.var_names
unique_sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].unique()
palette = sns.color_palette("viridis", len(unique_sample_ids))
sample_id_to_color = dict(zip(unique_sample_ids, palette))

for feature_name in var_names:
    # Calculate global min and max for the feature
    all_feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X.flatten()
    global_min = np.nanmin(all_feature_data)
    global_max = np.nanmax(all_feature_data)

    # Second pass: plot each lineage with the global min/max
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))  # Wide and short plots
    axes = axes.flatten()
    subplot_index = 0

    for lineage_number in pseudotime_cutoffs:
        pseudotime_col = f'Pseudotime_Lineage{lineage_number}'
        if pseudotime_col not in standard_trimmed_noPSTAT5_adata_sub.obs or subplot_index >= 3:
            continue

        # Extracting data for plotting
        pseudotime = standard_trimmed_noPSTAT5_adata_sub.obs[pseudotime_col].to_numpy().flatten()
        feature_data = standard_trimmed_noPSTAT5_adata_sub[:, feature_name].X
        sample_ids = standard_trimmed_noPSTAT5_adata_sub.obs['sample_ID'].to_numpy().flatten()  # Use sample_ID

        if not isinstance(feature_data, np.ndarray):
            feature_data = feature_data.toarray()
        feature_data = feature_data.flatten()

        max_pt = pseudotime_cutoffs[lineage_number]  # Get the cutoff for the current lineage
        valid_indices = ~np.isnan(pseudotime) & ~np.isnan(feature_data) & (pseudotime <= max_pt if max_pt is not None else True)
        pseudotime_valid = pseudotime[valid_indices]
        feature_data_valid = feature_data[valid_indices]
        sample_ids_valid = sample_ids[valid_indices]

        if len(pseudotime_valid) > 0 and len(feature_data_valid) > 0:
            z = np.polyfit(pseudotime_valid, feature_data_valid, 3)
            p = np.poly1d(z)
            pseudotime_sorted = np.sort(pseudotime_valid)
            plt_poly = p(pseudotime_sorted)

            # Plot data and polyfit
            scatter = axes[subplot_index].scatter(pseudotime_valid, feature_data_valid, alpha=0.5, 
                                                  c=[sample_id_to_color[id] for id in sample_ids_valid])
            axes[subplot_index].plot(pseudotime_sorted, plt_poly, color='red')
            axes[subplot_index].set_ylim(global_min, global_max)
            axes[subplot_index].set_xlabel(pseudotime_col)
            axes[subplot_index].set_ylabel(feature_name)

            subplot_index += 1

    plt.tight_layout()
    
    # Save the plot to a BytesIO object
    img_data = io.BytesIO()
    plt.savefig(img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    img_data.seek(0)
    plt.close()

    # Create and save the legend
    fig_legend, ax_legend = plt.subplots(figsize=(2, 2))
    ax_legend.axis('off')
    legend_handles = [plt.Line2D([0], [0], marker='o', color=sample_id_to_color[id], linestyle='', label=id) for id in unique_sample_ids]
    ax_legend.legend(handles=legend_handles, title="Sample IDs", loc='center', fontsize='small', frameon=False)
    legend_img_data = io.BytesIO()
    plt.savefig(legend_img_data, format='png', bbox_inches='tight', pad_inches=0.1)
    legend_img_data.seek(0)
    plt.close()

    # Add the plot and the legend to the PowerPoint slide
    add_slide_with_plot(prs, img_data, f'{feature_name} expression', legend_img_data)

# Save the PowerPoint presentation
prs.save('feature_plots.pptx')
